In [7]:
import torch
import numpy as np
from torch.utils.data import TensorDataset
import pandas as pd
import json

In [8]:
test = pd.read_csv("dataset/test_preprocessed.csv")
train = pd.read_csv("dataset/train_preprocessed.csv")

In [9]:
# dropColumns = [
#     "SERIAL",
#     "PERNUM",
#     "HHWT",
#     "CLUSTER",
#     "STRATA",
#     "PERWT",
#     "RACNUM",
#     "YRMARR",
#     "YRNATUR",
#     "WKSWORK1",
#     "UHRSWORK",
#     "TRANTIME",
#     "index",
#     "carpools",
#     "has"
    
# ]

# test = test.drop(columns=dropColumns)
# train = train.drop(columns=dropColumns)


for column in test.columns:
    if(column.find("AGE") != -1):
        continue
    if(column.find("hasDegree") != -1):
        continue
    if(column.find("worksIn") != -1):
        continue
    if(column.find("occupation") != -1):
        continue

    test = test.drop(columns=column)
    
for column in train.columns:
    if(column.find("AGE") != -1):
        continue
    if(column.find("hasDegree") != -1):
        continue
    if(column.find("worksIn") != -1):
        continue
    if(column.find("occupation") != -1):
        continue
    
    train = train.drop(columns=column)

for column in test.columns:
	test[column].fillna(False, inplace=True)
 
for column in train.columns:
	train[column].fillna(False, inplace=True)


In [10]:
for column in test.columns:
	test[column] = test[column].astype(np.float32)

for column in train.columns:
	train[column] = train[column].astype(np.float32)

with open("dataset/columns.json", "w") as f:
	json.dump(test.columns.tolist(), f)
 


In [11]:
salaryShift = 50000

xTrainTensor = torch.tensor(train.drop(columns=['INCWAGE_CPIU_2010']).values, dtype=torch.float32)
yTrainTensor = torch.tensor((train['INCWAGE_CPIU_2010']-salaryShift).values, dtype=torch.float32)

xTestTensor = torch.tensor(test.drop(columns=['INCWAGE_CPIU_2010']).values, dtype=torch.float32)
yTestTensor = torch.tensor((test['INCWAGE_CPIU_2010']-salaryShift).values, dtype=torch.float32)

In [12]:
print(xTrainTensor.shape)
print(yTrainTensor.shape)
print(xTestTensor.shape)
print(yTestTensor.shape)

torch.save(xTrainTensor, 'dataset/trainSetX.pt')
torch.save(yTrainTensor, 'dataset/trainSetY.pt')

torch.save(xTestTensor, 'dataset/testSetX.pt')
torch.save(yTestTensor, 'dataset/testSetY.pt')

torch.Size([1231497, 517])
torch.Size([1231497])
torch.Size([217324, 517])
torch.Size([217324])
